# AskAPI

v0.1 - unstable

Lets you interact with any API using natural language.

In [ ]:
!pip install openai requests python-docx beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os
import requests
import json
from bs4 import BeautifulSoup
from docx import Document
from openai import OpenAI

Provide API key to https://api.aimlapi.com

In [ ]:
# Initialize AIMLAPI Client (use your own API key and endpoint)
client = OpenAI(
    api_key="...",
    base_url="https://api.aimlapi.com",
)

def fetch_documentation_from_url(url):
    """Fetch API documentation from a URL and extract plain text."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text(separator='\n')
    return text

def fetch_documentation_from_docx(filepath):
    """Fetch API documentation from a .docx file."""
    doc = Document(filepath)
    text = '\n'.join([para.text for para in doc.paragraphs])
    return text

def query_gpt(messages):
    """Send messages to the AIMLAPI GPT model and return the response."""
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        max_tokens = 10000
    )
    return response.choices[0].message.content.strip()

def build_knowledge_base(doc_text):
    """Use GPT to generate a structured knowledge base from the provided documentation text."""
    messages = [
        {"role": "system", "content": "You are an AI assistant who translates API documentation into a knowledge base. Your reply is always in JSON format only, no other comments."},
        {"role": "user", "content": f"Analyze the following API documentation and extract the base URL, request methods, parameters, and authentication methods in JSON format:\n\n{doc_text}"}
    ]

    # Get response from GPT
    knowledge_base_str = query_gpt(messages)

    # Debugging: Print the raw GPT response to ensure it's in JSON format
    #print(f"Raw GPT Response: {knowledge_base_str}")

    # Remove code block markers if they exist
    if knowledge_base_str.startswith('```json'):
        knowledge_base_str = knowledge_base_str[7:]  # Remove the first 7 characters: ```json
    if knowledge_base_str.endswith('```'):
        knowledge_base_str = knowledge_base_str[:-3]  # Remove the last 3 characters: ```

    # Debugging: Print the cleaned-up knowledge base string
    print(f"Cleaned GPT Response: {knowledge_base_str}")

    # Try to parse the cleaned-up response as JSON
    try:
        knowledge_base = json.loads(knowledge_base_str)

        # Debugging: Print parsed knowledge base
        print(f"Parsed Knowledge Base: {json.dumps(knowledge_base, indent=2)}")

        if 'base_url' not in knowledge_base:
            print("Error: 'base_url' is missing from the knowledge base after parsing.")
            return {}

    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        knowledge_base = {}

    return knowledge_base

def extract_parameters(user_query, knowledge_base):
    """Use GPT to extract relevant API parameters based on the user's query."""

    # Debugging: Print the user query and knowledge base before sending to GPT
    print(f"User Query: {user_query}")
    print(f"Knowledge Base: {json.dumps(knowledge_base, indent=2)}")

    # Refined prompt to request proper output structure (e.g., query_params)
    messages = [
        {"role": "system", "content": "You are an AI assistant who translates user query to API parameters."},
        {"role": "user", "content": f"""Given the following user query: '{user_query}', and API knowledge base: <knowledge>{json.dumps(knowledge_base, indent=2)}</knowledge>, extract the necessary API parameters in JSON format. The JSON should contain:
        1. 'query_params' object with the exact parameters required for the API call.
        2. If applicable, include 'latitude' and 'longitude' instead of just location name (use New York as an example: 'latitude': 40.7128, 'longitude': -74.0060).
        3. Include date information such as 'yesterday' translated into an actual date (e.g., '2023-10-21' if today is 2023-10-22).
        4. Output only JSON format, no comments."""}
    ]

    params_text = query_gpt(messages)

    # Debugging: Print the raw GPT response
    print(f"Raw GPT Response: {params_text}")

    try:
        # Ensure that we only return valid query_params from the response
        params = json.loads(params_text)

        # Debugging: Print the parsed parameters
        print(f"Extracted Parameters: {json.dumps(params, indent=2)}")

        # Check if 'query_params' is in the response
        if "query_params" not in params:
            print("Error: 'query_params' not found in the response. Returning empty parameters.")
            return {}

    except json.JSONDecodeError:
        print("Error: Could not parse the parameters. Returning empty parameters.")
        params = {}

    return params

def create_api_call(knowledge_base, params):
    """Create the full API call based on the knowledge base and extracted parameters."""

    # Debugging: Ensure the base_url is available
    base_url = knowledge_base.get('base_url')

    if not base_url:
        print("Error: 'base_url' is missing from the knowledge base. Cannot make API call.")
        return {"error": "Missing base_url"}

    method = knowledge_base.get('method', 'GET').upper()
    headers = knowledge_base.get('headers', {})
    query_params = params.get('query_params', {})
    body = params.get('body', {})

    # Debugging: Print the API details before making the call
    print(f"Making {method} request to {base_url} with params: {query_params} and body: {body}")

    # Handle GET and POST requests based on the method specified
    if method == 'GET':
        api_url = base_url
        response = requests.get(api_url, headers=headers, params=query_params)
    elif method == 'POST':
        api_url = base_url
        response = requests.post(api_url, headers=headers, json=body)
    else:
        return {"error": "Unsupported request method."}

    return response.json()

def summarize_response(api_response):
    """Summarize the API response into natural language using GPT."""
    messages = [
        {"role": "system", "content": "You are an AI assistant who can summarize any API response and explain it in plain, clearly understandable language."},
        {"role": "user", "content": f"Summarize the following API response:\n\n{json.dumps(api_response, indent=2)}"}
    ]
    summary = query_gpt(messages)
    return summary

In [ ]:
# Step 1: Load API documentation
source = input("Load API documentation from (url/docx): ").strip().lower()
if source == 'url':
    # for example: https://open-meteo.com/en/docs
    url = input("Enter the API documentation URL: ").strip()
    doc_text = fetch_documentation_from_url(url)
elif source == 'docx':
    filepath = input("Enter the path to the .docx file: ").strip()
    doc_text = fetch_documentation_from_docx(filepath)
else:
    print("Invalid source.")

Load API documentation from (url/docx): url
Enter the API documentation URL: https://open-meteo.com/en/docs


In [ ]:
# Step 2: Build knowledge base from the documentation
print("Building knowledge base...")
knowledge_base = build_knowledge_base(doc_text)
print("Knowledge base built.")


Building knowledge base...
Raw GPT Response: ```json
{
  "base_url": "https://api.open-meteo.com/v1/forecast",
  "request_methods": [
    "GET"
  ],
  "parameters": [
    {
      "name": "latitude",
      "type": "Floating point",
      "required": true,
      "default": "",
      "description": "Geographical WGS84 coordinates of the location."
    },
    {
      "name": "longitude",
      "type": "Floating point",
      "required": true,
      "default": "",
      "description": "Geographical WGS84 coordinates of the location."
    },
    {
      "name": "elevation",
      "type": "Floating point",
      "required": false,
      "default": "90",
      "description": "The elevation used for statistical downscaling."
    },
    {
      "name": "hourly",
      "type": "String array",
      "required": false,
      "default": "",
      "description": "A list of weather variables which should be returned."
    },
    {
      "name": "daily",
      "type": "String array",
      "required": 

In [ ]:
# Input a user query
user_query = "yesterday temperature in New York"  # Modify as needed

# Extract parameters based on the query and knowledge base
params = extract_parameters(user_query, knowledge_base)
if params:
    print("Parameters extracted successfully.")
else:
    print("Failed to extract parameters.")


User Query: yesterday temperature in New York
Knowledge Base: {
  "base_url": "https://api.open-meteo.com/v1/forecast",
  "request_methods": [
    "GET"
  ],
  "parameters": [
    {
      "name": "latitude",
      "type": "Floating point",
      "required": true,
      "default": "",
      "description": "Geographical WGS84 coordinates of the location."
    },
    {
      "name": "longitude",
      "type": "Floating point",
      "required": true,
      "default": "",
      "description": "Geographical WGS84 coordinates of the location."
    },
    {
      "name": "elevation",
      "type": "Floating point",
      "required": false,
      "default": "90",
      "description": "The elevation used for statistical downscaling."
    },
    {
      "name": "hourly",
      "type": "String array",
      "required": false,
      "default": "",
      "description": "A list of weather variables which should be returned."
    },
    {
      "name": "daily",
      "type": "String array",
      "re

In [ ]:
# Make the API call and get the response
api_response = create_api_call(knowledge_base, params)

# Print the raw API response
print("API Response:", api_response)


Making GET request to https://api.open-meteo.com/v1/forecast with params: {'latitude': 40.7128, 'longitude': -74.006, 'daily': ['temperature_2m_max', 'temperature_2m_min'], 'timezone': 'GMT', 'past_days': 1} and body: {}
API Response: {'latitude': 40.710335, 'longitude': -73.99309, 'generationtime_ms': 0.06699562072753906, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 32.0, 'daily_units': {'time': 'iso8601', 'temperature_2m_max': '°C', 'temperature_2m_min': '°C'}, 'daily': {'time': ['2024-10-07', '2024-10-08', '2024-10-09', '2024-10-10', '2024-10-11', '2024-10-12', '2024-10-13', '2024-10-14'], 'temperature_2m_max': [24.9, 20.5, 18.9, 16.5, 20.0, 24.3, 20.9, 22.3], 'temperature_2m_min': [17.6, 10.8, 11.4, 8.6, 8.8, 14.5, 12.9, 13.8]}}


In [ ]:
# Summarize the API response
summary = summarize_response(api_response)
print("Summary:", summary)


Summary: The API response provides weather forecast data for a specific location with the following details:

- **Location Coordinates:** Latitude 40.710335 and Longitude -73.99309.
- **Timezone:** The data is in GMT (Greenwich Mean Time).
- **Elevation:** The location is at an elevation of 32 meters above sea level.
- **Forecast Period:** The forecast covers dates from October 7 to October 14, 2024.

**Daily Temperature Forecast:**
- **Maximum Temperatures (°C):** 
  - October 7: 24.9°C
  - October 8: 20.5°C
  - October 9: 18.9°C
  - October 10: 16.5°C
  - October 11: 20.0°C
  - October 12: 24.3°C
  - October 13: 20.9°C
  - October 14: 22.3°C

- **Minimum Temperatures (°C):** 
  - October 7: 17.6°C
  - October 8: 10.8°C
  - October 9: 11.4°C
  - October 10: 8.6°C
  - October 11: 8.8°C
  - October 12: 14.5°C
  - October 13: 12.9°C
  - October 14: 13.8°C

In summary, the forecast shows a range of temperatures over the specified dates, with the highest temperatures occurring in the begin